In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
import matplotlib as mpl
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import scipy.stats as stats
import scipy.optimize as opt
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
run = 72 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'


saveFig = False
campaign = 'new_normScan/dataCleaning'

dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)




Make spectra of various energy parameters for muon cut

In [ ]:
# Make energy spectrum
fig, ax = plt.subplots()
raw_elo, raw_ehi, raw_epb = 0, 11000000, 5000
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10


        

nbx = int((raw_ehi-raw_elo)/raw_epb)

raw_energy_hist, ebins = np.histogram(df_raw['energy'], bins=nbx,
                range=[raw_elo, raw_ehi])


plt.semilogy(ebins[1:], raw_energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


plt.xlabel('on-board energy (ADC)', fontsize=16)
plt.ylabel('counts', fontsize=16)
plt.title('Raw Energy', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)
if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_rawE.png', dpi=200)

In [ ]:
# Make energy spectrum
fig, ax = plt.subplots()
raw_elo, raw_ehi, raw_epb = 50, 17000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10


        

nbx = int((raw_ehi-raw_elo)/raw_epb)

raw_energy_hist, ebins = np.histogram(df_raw['wf_max'], bins=nbx,
                range=[raw_elo, raw_ehi])


plt.semilogy(ebins[1:], raw_energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


plt.xlabel('WF max', fontsize=16)
plt.ylabel('counts', fontsize=16)
plt.title('Waveform Max', fontsize=16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_wfMax.png', dpi=200)

In [ ]:
# Make energy spectrum
fig, ax = plt.subplots()
raw_elo, raw_ehi, raw_epb = 50, 17000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10


        

nbx = int((raw_ehi-raw_elo)/raw_epb)

raw_energy_hist, ebins = np.histogram(df_raw['trapEmax'], bins=nbx,
                range=[raw_elo, raw_ehi])


plt.semilogy(ebins[1:], raw_energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


plt.xlabel('trapEmax', fontsize=16)
plt.ylabel('counts', fontsize=16)
plt.title('trapEmax', fontsize=16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_trapEmax.png', dpi=200)

In [ ]:
# Make energy spectrum
fig, ax = plt.subplots()
elo, ehi, epb = 50, 6000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, ebins = np.histogram(df_raw['trapEftp_cal'], bins=nbx,
                range=[elo, ehi])


plt.semilogy(ebins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.xlabel('trapEftp_cal (keV)', fontsize=16)
plt.ylabel('counts', fontsize=16)
plt.title('Calibrated Energy', fontsize=16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_trapEftp_cal.png', dpi=200)

Muon cut

In [ ]:
muon_cut = f'energy < 11000000 and wf_max<17000 and trapEmax < 17000 and trapEftp_cal < 6000'
df_raw = df_raw.query(muon_cut).copy()

fit 60 keV peak to get its energy resolution

In [ ]:
# First use gauss_mode_width_max to use for initial guesses in fit_hist

pgfenergy_hist, pgfebins, evars = pgh.get_hist(df_raw['trapEftp_cal'], bins=50, range=[54, 65]) #range=[54, 65]
plt.plot(pgfebins[1:], pgfenergy_hist, ds='steps', c='b', lw=1)
pars, cov = pgf.gauss_mode_width_max(pgfenergy_hist, pgfebins, evars)
mode = pars[0]
width = pars[1]
amp = pars[2]
print(f'mode: {mode}')
print(f'width: {width}')
print(f'amp: {amp}')

In [ ]:
e_pars, ecov = pgf.fit_hist(cage_utils.gauss_fit_func, pgfenergy_hist, pgfebins, evars, guess = (amp, mode, width, 1))

mean_fit = e_pars[1]
width_fit = e_pars[2]
amp_fit = e_pars[0]
const_fit = e_pars[3]

fwhm = width_fit*2.355

print(f'mean: {mean_fit}')
print(f'width: {width_fit}')
print(f'amp: {amp_fit}')
print(f'C: {const_fit}')
print(f'FWHM at 60 keV: {fwhm} \n{(fwhm/mean_fit)*100}%')

In [ ]:
plt.plot(pgfebins[1:], cage_utils.gauss_fit_func(pgfebins[1:], *e_pars), c = 'r')
plt.plot(pgfebins[1:], pgfenergy_hist, ds='steps', c='b', lw=1)

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 0, 5500, 1
e_bins = 2000 #int((ehi-elo)/epb)
wflo, wfhi = 0, 15000
wf_bins = 2000
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
wf_maxVEnergy, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]


pcm = plt.pcolormesh(X, Y, wf_maxVEnergy,norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', position=(0.1, 0.85), transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)


# plt.ylim(0, 300)
# plt.xlim(0, 800)

# plt.ylim(500, 800)
# plt.xlim(1400, 2000)

# plt.ylim(1200, 1550)
# plt.xlim(3300, 4300)


plt.ylim(2400, 2750)
plt.xlim(6500, 8500)

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_wf_max_Vs_trapEftp_cal.png', dpi=200)

In [ ]:
stat_used_wf_max = 'median'

median, xedges, binnumber = stats.binned_statistic(df_raw['wf_max'], df_raw['trapEftp_cal'], statistic = stat_used_wf_max, bins = wf_bins) #, range = (wflo, wfhi)


en_bin_centers = pgh.get_bin_centers(xedges)
cal_en_bin_centers = pgh.get_bin_centers(yedges)

idx = np.isfinite(en_bin_centers) & np.isfinite(median)


fit_raw, cov = np.polyfit(en_bin_centers[idx], median[idx], deg=1, cov=True)

const = fit_raw[0]
offset = fit_raw[1]
err = np.sqrt(np.diag(cov))

In [ ]:
e_res_const = [0., 0., 0.]
with open('./cuts.json', 'r+') as f:
    cuts = json.load(f)
    e_res_const[0] = cuts[str(run)]['e_res_const0']
    e_res_const[1] = cuts[str(run)]['e_res_const1']
    e_res_const[2] = cuts[str(run)]['e_res_const2']
    
print(e_res_const)

In [ ]:
fig, ax = plt.subplots()


elo, ehi, epb = 0, 5500, 1
e_bins = 2000 #int((ehi-elo)/epb)
wflo, wfhi = 0, 15000
wf_bins = 2000 #int((wfhi - wflo )/10)
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm()) #, norm=LogNorm()
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 10})

ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(200, 1000)
# plt.ylim(0, 400)

y = (const*en_bin_centers + offset)
plt.plot(en_bin_centers, y, 'r', label='fit')

# z = const*en_bin_centers + offset*(1-0.04*cal_en_bin_centers)
# plt.plot(en_bin_centers, z, 'g')

# w = const*en_bin_centers + offset*(1+0.04*cal_en_bin_centers) #en_bin_centers*0.04
# plt.plot(en_bin_centers, w, 'b')


# y = (const*en_bin_centers + offset)
# plt.plot(en_bin_centers, y, 'r')

z = (const*en_bin_centers + 
offset + 2.*np.sqrt(e_res_const[0]+e_res_const[1]*cal_en_bin_centers +
            (e_res_const[2]*cal_en_bin_centers**2))
    ) 

plt.plot(en_bin_centers, z, 'c', label= 'cut_hi')

w = (const*en_bin_centers + 
offset - 2.*np.sqrt(e_res_const[0]+e_res_const[1]*cal_en_bin_centers +
            e_res_const[2]*cal_en_bin_centers**2)
    )
plt.plot(en_bin_centers, w, 'b', label='cut_lo')

# plt.ylim(2000, 5000)
# plt.ylim(40, 80)
# plt.xlim(110, 250)

plt.ylim(0, 300)
plt.xlim(0, 800)

# plt.ylim(500, 800)
# plt.xlim(1400, 2000)

# plt.ylim(1200, 1550)
# plt.xlim(3300, 4300)


# plt.ylim(2400, 2750)
# plt.xlim(6500, 8500)
plt.legend(fontsize=14)

In [ ]:
wf_max_cut = f'({const}*wf_max + {offset} - 2.*sqrt({e_res_const[0]}+{e_res_const[1]}*trapEftp_cal + ({e_res_const[2]}*trapEftp_cal**2)) ) < trapEftp_cal < ({const}*wf_max + {offset} + 2.*sqrt({e_res_const[0]}+{e_res_const[1]}*trapEftp_cal + ({e_res_const[2]}*trapEftp_cal**2)) )'


df_raw_tot = len(df_raw)
wf_max_cut_pass = len(df_raw.query(wf_max_cut).copy())
wf_max_pass_percent = (wf_max_cut_pass/df_raw_tot)*100
print(f'{wf_max_cut_pass} events passing cut \n{wf_max_pass_percent:.2f}%')

df= df_raw.query(wf_max_cut).copy()

# print(wf_max_cut)

If cut looks good, save plot and include % events passing cut

In [ ]:
fig, ax = plt.subplots()


elo, ehi, epb = 0, 5500, 1
e_bins = 2000 #int((ehi-elo)/epb)
wflo, wfhi = 0, 15000
wf_bins = 2000 #int((wfhi - wflo )/10)
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm()) #, norm=LogNorm()
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f} \n{wf_max_pass_percent:.2f}% passing cut', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 10})

ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(200, 1000)
# plt.ylim(0, 400)

y = (const*en_bin_centers + offset)
plt.plot(en_bin_centers, y, 'r', label='fit')

z = (const*en_bin_centers + 
offset + 2.*np.sqrt(e_res_const[0]+e_res_const[1]*cal_en_bin_centers +
            (e_res_const[2]*cal_en_bin_centers**2))
    ) 

plt.plot(en_bin_centers, z, 'c', label= 'cut_hi')

w = (const*en_bin_centers + 
offset - 2.*np.sqrt(e_res_const[0]+e_res_const[1]*cal_en_bin_centers +
            e_res_const[2]*cal_en_bin_centers**2)
    )
plt.plot(en_bin_centers, w, 'b', label='cut_lo')

# plt.ylim(2000, 5000)
# plt.ylim(40, 80)
# plt.xlim(110, 250)

# plt.ylim(40, 80)
# plt.xlim(50, 300)

plt.ylim(0, 300)
plt.xlim(0, 800)

# plt.ylim(500, 800)
# plt.xlim(1400, 2000)

# plt.ylim(1200, 1550)
# plt.xlim(3300, 4300)


# plt.ylim(2400, 2750)
# plt.xlim(6500, 8500)
plt.legend(fontsize=14)
if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_lowEfit_wf_max_Vs_trapEftp_cal.png', dpi=200)

Now look at parameters after basic cuts

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 10, 100, 0.5
e_bins = int((ehi-elo)/epb)
wflo, wfhi = 10, 500
wf_bins = int((wfhi - wflo )/10)
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df['wf_max'], df['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f} \n{wf_max_pass_percent:.2f}% passing cut', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(200, 1000)
# plt.ylim(0, 400)
y = (const*en_bin_centers + offset)
plt.plot(en_bin_centers, y, 'r', label='fit')

z = (const*en_bin_centers + 
offset + 2.*np.sqrt(e_res_const[0]+e_res_const[1]*cal_en_bin_centers +
            (e_res_const[2]*cal_en_bin_centers**2))
    ) 

plt.plot(en_bin_centers, z, 'c', label= 'cut_hi')

w = (const*en_bin_centers + 
offset - 2.*np.sqrt(e_res_const[0]+e_res_const[1]*cal_en_bin_centers +
            e_res_const[2]*cal_en_bin_centers**2)
    )
plt.plot(en_bin_centers, w, 'b', label='cut_lo')

# plt.ylim(2000, 5000)
plt.ylim(10, 100)
plt.xlim(50, 500)
plt.legend(fontsize=14)
plt.show()

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 10, 100, 0.5
e_bins = int((ehi-elo)/epb)
trapEmax_lo, trapEmax_hi = 10, 350
trapEmax_bins = int((wfhi - wflo )/10)
fig.suptitle(f'\nTrapEmax vs Energy', horizontalalignment='center', fontsize=16)
wf_maxVEnergy, xedges, yedges = np.histogram2d(df['trapEmax'], df['trapEftp_cal'], bins=[trapEmax_bins, e_bins], range=([trapEmax_lo, trapEmax_hi], [elo, ehi]))
X, Y = np.mgrid[trapEmax_lo:trapEmax_hi:trapEmax_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, wf_maxVEnergy, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('trapEmax (uncal)', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)

plt.show()

Baseline cuts

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_raw = np.quantile(df_raw['bl'], q = (0.03, 0.98)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_raw)

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 9000,9400, 1
nbx = int((bhi-blo)/bpb)

# bl_cut_lo, bl_cut_hi = 9160,9295 # run 70
# bl_cut_lo, bl_cut_hi = 9225,9310
bl_cut_lo_raw, bl_cut_hi_raw = quant_raw[0], quant_raw[1]

bl_hist, bins = np.histogram(df_raw['bl'], bins=nbx,
                range=[blo, bhi])

plt.plot(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_cut_lo_raw, c='r', lw=1)
plt.axvline(bl_cut_hi_raw, c='r', lw=1)

plt.xlabel('bl', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('Baseline Mean before cuts\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_bl_mean_raw.png', dpi=200)

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant = np.quantile(df['bl'], q = (0.03, 0.98)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant)

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 9000,9400, 1
nbx = int((bhi-blo)/bpb)

# bl_cut_lo, bl_cut_hi = 9160,9295 # run 70
# bl_cut_lo, bl_cut_hi = 9225,9310
bl_cut_lo, bl_cut_hi = quant_raw[0], quant_raw[1]

bl_hist, bins = np.histogram(df['bl'], bins=nbx,
                range=[blo, bhi])

plt.plot(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('Baseline Mean after wf_max cut\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_bl_mean.png', dpi=200)

In [ ]:
bl_mean_cut_raw = f'bl > {bl_cut_lo_raw} and bl < {bl_cut_hi_raw}'
bl_mean_cut = f'bl > {bl_cut_lo} and bl < {bl_cut_hi}'

Baseline slope

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_raw_slope = np.quantile(df_raw['bl_slope'], q = (0.02, 0.97)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_raw_slope)

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = -10., 10., 0.005
nbx = int((bhi-blo)/bpb)

# bl_slope_lo, bl_slope_hi = -4.5, 4.5 # run 70
# bl_slope_lo, bl_slope_hi = -5.0, 5.
bl_slope_lo_raw, bl_slope_hi_raw = quant_raw_slope[0], quant_raw_slope[1]

bl_hist, bins = np.histogram(df_raw['bl_slope'], bins=nbx,
                range=[blo, bhi])

plt.plot(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_slope_lo_raw, c='r', lw=1)
plt.axvline(bl_slope_hi_raw, c='r', lw=1)

plt.xlabel('bl_slope', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('Baseline Slope before cuts\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_bl_slope_raw.png', dpi=200)

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_slope = np.quantile(df['bl_slope'], q = (0.02, 0.97)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_slope)

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = -10., 10., 0.005
nbx = int((bhi-blo)/bpb)

# bl_slope_lo, bl_slope_hi = -4.5, 4.5 # run 70
# bl_slope_lo, bl_slope_hi = -5.0, 5.
bl_slope_lo, bl_slope_hi = quant_slope[0], quant_slope[1]

bl_hist, bins = np.histogram(df['bl_slope'], bins=nbx,
                range=[blo, bhi])

plt.plot(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_slope_lo, c='r', lw=1)
plt.axvline(bl_slope_hi, c='r', lw=1)

plt.xlabel('bl_slope', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('Baseline Slope after wf_max cut\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_bl_slope.png', dpi=200)

In [ ]:
bl_slope_cut_raw = f'bl_slope > {bl_slope_lo_raw} and bl_slope < {bl_slope_hi_raw}'
bl_slope_cut = f'bl_slope > {bl_slope_lo} and bl_slope < {bl_slope_hi}'

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_raw_sig = np.quantile(df_raw['bl_sig'], q = (0.001, 0.951)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_raw_sig)

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 2., 12., 0.005
nbx = int((bhi-blo)/bpb)

# bl_sig_lo, bl_sig_hi = 3.1, 5. # run 70
# bl_sig_lo, bl_sig_hi = 3.1, 5.25
bl_sig_lo_raw, bl_sig_hi_raw = quant_raw_sig[0], quant_raw_sig[1]  

bl_hist, bins = np.histogram(df_raw['bl_sig'], bins=nbx,
                range=[blo, bhi])

plt.plot(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_sig_lo_raw, c='r', lw=1)
plt.axvline(bl_sig_hi_raw, c='r', lw=1)

plt.xlabel('bl_sigma', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('Baseline Sigma before cuts\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_bl_sig_raw.png', dpi=200)

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_sig = np.quantile(df['bl_sig'], q = (0.001, 0.951)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_sig)

In [ ]:
fig, ax = plt.subplots()
blo, bhi, bpb = 2., 12., 0.005
nbx = int((bhi-blo)/bpb)

# bl_sig_lo, bl_sig_hi = 3.1, 5. # run 70
# bl_sig_lo, bl_sig_hi = 3.1, 5.25
bl_sig_lo, bl_sig_hi = quant_sig[0], quant_sig[1]  

bl_hist, bins = np.histogram(df['bl_sig'], bins=nbx,
                range=[blo, bhi])

plt.plot(bins[1:], bl_hist, ds='steps', c='b', lw=1)


plt.axvline(bl_sig_lo, c='r', lw=1)
plt.axvline(bl_sig_hi, c='r', lw=1)

plt.xlabel('bl_sigma', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('Baseline Sigma after wf_max cut\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_bl_sig.png', dpi=200)

In [ ]:
bl_sig_cut_raw = f'bl_sig > {bl_sig_lo_raw} and bl_sig < {bl_sig_hi_raw}'
bl_sig_cut = f'bl_sig > {bl_sig_lo} and bl_sig < {bl_sig_hi}'

trapEftp vs trapEmax cuts

In [ ]:
df_raw['ftp_max'] = df_raw['trapEftp']/df_raw['trapEmax']
df['ftp_max'] = df['trapEftp']/df['trapEmax']

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_ftp_max_raw = np.quantile(df_raw['ftp_max'], q = (0.049, 0.999)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_ftp_max_raw)

In [ ]:
fig, ax = plt.subplots()

elo, ehi = 0.925, 1.01
e_bins = int((ehi - elo )/0.001)

ftp_max_lo_raw, ftp_max_hi_raw = quant_ftp_max_raw[0], quant_ftp_max_raw[1]

ftp_max_hist, bins = np.histogram(df_raw['ftp_max'], bins=nbx,
                range=[elo, ehi])

plt.plot(bins[1:], ftp_max_hist, ds='steps', c='b', lw=1)


plt.axvline(ftp_max_lo_raw, c='r', lw=1)
plt.axvline(ftp_max_hi_raw, c='r', lw=1)

plt.xlabel('trapEftp/trapEmax', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('trapEftp/trapEmax before cuts\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', position = (0.1, 0.85), transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_ftp_max_raw.png', dpi=200)

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_ftp_max = np.quantile(df['ftp_max'], q = (0.049, 0.999)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_ftp_max)

In [ ]:
fig, ax = plt.subplots()
elo, ehi = 0.925, 1.01
e_bins = int((ehi - elo )/0.001)

ftp_max_lo, ftp_max_hi = quant_ftp_max[0], quant_ftp_max[1]

ftp_max_hist, bins = np.histogram(df['ftp_max'], bins=nbx,
                range=[elo, ehi])

plt.plot(bins[1:], ftp_max_hist, ds='steps', c='b', lw=1)


plt.axvline(ftp_max_lo, c='r', lw=1)
plt.axvline(ftp_max_hi, c='r', lw=1)

plt.xlabel('trapEftp/trapEmax', fontsize=16)
plt.ylabel('counts', fontsize=16)

plt.title('trapEftp/trapEmax after wf_max cut\nwith 95% cut lines', fontsize = 16)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nruntime {rt_min:.2f}', position = (0.1, 0.85), transform=ax.transAxes, color='black', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

if saveFig:
    plt.savefig(f'./plots/{campaign}/run{run}_ftp_max.png', dpi=200)

In [ ]:
ftp_max_cut_raw = f'ftp_max  > {ftp_max_lo_raw} and ftp_max < {ftp_max_hi_raw}'
ftp_max_cut = f'ftp_max  > {ftp_max_lo} and ftp_max < {ftp_max_hi}'

Write cuts to json file:

In [ ]:
#Cuts to write:
# write_cuts = True
write_cuts = False

cut_keys = ['muon_cut', 'wf_max_cut', 'bl_mean_cut_raw', 'bl_mean_cut', 'bl_slope_cut_raw', 'bl_slope_cut', 
            'bl_sig_cut_raw', 'bl_sig_cut', 'ftp_max_cut_raw', 'ftp_max_cut', 'stat_used_wf_max']
cut_list = [muon_cut, wf_max_cut, bl_mean_cut_raw, bl_mean_cut, bl_slope_cut_raw, bl_slope_cut, 
            bl_sig_cut_raw, bl_sig_cut, ftp_max_cut_raw, ftp_max_cut, stat_used_wf_max]


if write_cuts == True:
    for key, cut in zip(cut_keys, cut_list):
        cage_utils.writeCuts(run, key, cut)
        
        
    


    

Look at these cuts after basic cuts in the future

T/E

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_raw_ToE = np.quantile(df_raw['ToE'], q = (0.001, 0.951)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_raw_ToE)

In [ ]:
ToElo, ToEhi, ToEpb= 0.1,  0.5, 0.0025
nbx = int((ToEhi-ToElo)/ToEpb)

# bl_sig_lo, bl_sig_hi = 3.1, 5. # run 70
# bl_sig_lo, bl_sig_hi = 3.1, 5.25
# bl_sig_lo_raw, bl_sig_hi_raw = quant_raw_sig[0], quant_raw_sig[1]  

ToE_hist, bins = np.histogram(df_raw['ToE'], bins=nbx,
                range=[ToElo, ToEhi])

plt.plot(bins[1:], ToE_hist, ds='steps', c='b', lw=1)


# plt.axvline(bl_sig_lo_raw, c='r', lw=1)
# plt.axvline(bl_sig_hi_raw, c='r', lw=1)

plt.xlabel('ToE', fontsize=16)
plt.ylabel('counts', fontsize=16)
plt.show()

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 40, 80, 0.5
ToElo, ToEhi, ToEpb= 0.,  0.5, 0.0025

nbx = int((ehi-elo)/epb)
nby = int((ToEhi-ToElo)/ToEpb)



fig.suptitle(f'T/E vs Energy', horizontalalignment='center', fontsize=16)

ToE_hist_norm, xedges, yedges = np.histogram2d(df_raw['trapEftp_cal'], df_raw['ToE'], bins=[nbx, nby], range=([elo, ehi], [ToElo, ToEhi]))
X, Y = np.mgrid[elo:ehi:nbx*1j, ToElo:ToEhi:nby*1j]


pcm = plt.pcolormesh(X, Y, ToE_hist_norm, norm=LogNorm()) #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'Energy (keV)', fontsize=16)
ax.set_ylabel('T/E (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg \nwf_triangle 100 ns, 10 ns', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'\n{runtype} run {run}, {rt_min:.2f} mins', fontsize=12)
plt.tight_layout()
# plt.ylim(-0.05, 0.1)
# plt.ylim(-0.05, 0.08)
# plt.ylim(-0.2, 0.2)
# plt.xlim(52.5, 65)
plt.show()

Wf_argmax cuts. Don't think I'll use this for now. 

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_raw_wf_argmax = np.quantile(df_raw['wf_argmax'], q = (0.001, 0.949)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_raw_wf_argmax)

In [ ]:
# Make energy spectrum
wflo, wfhi, wfpb = 35000, 50000, 100
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((wfhi-wflo)/wfpb)

wf_argmax_raw_lo, wf_argmax_raw_hi = quant_raw_wf_argmax[0], quant_raw_wf_argmax[1]  

wf_argmax_hist, bins = np.histogram(df_raw['wf_argmax'], bins=nbx,
                range=[wflo, wfhi])

plt.plot(bins[1:], wf_argmax_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.axvline(wf_argmax_raw_lo, c='r', lw=1)
plt.axvline(wf_argmax_raw_hi, c='r', lw=1)

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# # plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.xlim(80000, 150000)
plt.show()

In [ ]:
# quant = np.quantile(df_raw['bl'], q = (0.07, 0.97)) # 90%
quant_wf_argmax = np.quantile(df['wf_argmax'], q = (0.015, 0.965)) # 95%

# quant = np.quantile(bins, q = (0.5))

print(quant_wf_argmax)

In [ ]:
# Make energy spectrum
wflo, wfhi, wfpb = 35000, 50000, 50
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((wfhi-wflo)/wfpb)

wf_argmax_lo, wf_argmax_hi = quant_wf_argmax[0], quant_raw_wf_argmax[1]  

wf_argmax_hist, bins = np.histogram(df['wf_argmax'], bins=nbx,
                range=[wflo, wfhi])

plt.plot(bins[1:], wf_argmax_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.axvline(wf_argmax_lo, c='r', lw=1)
plt.axvline(wf_argmax_hi, c='r', lw=1)

# plt.xlabel(etype, ha='right', x=1)
# plt.ylabel('cts / sec', ha='right', y=1)
# # plt.legend()
# # plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
# plt.xlim(80000, 150000)
plt.show()